# Business Dataset

In [78]:
from pathlib import Path
import pandas as pd
import json
import numpy as np

RAW = Path("../data/raw")

# 1) Read JSON Lines
biz = pd.read_json(RAW / "yelp_academic_dataset_business.json", lines=True)

print(biz.shape)       # rows, columns
print(biz.columns)  # column names
biz.info()

(150346, 14)
Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hou

In [79]:
# Ensure review_count is numeric
biz["review_count"] = pd.to_numeric(biz["review_count"], errors="coerce").astype("float32")

# Log transform review_count to reduce skewness
biz["review_count_log1p"] = np.log1p(biz["review_count"]).astype("float32")

# --- Helpers --------------------------------------------------------------- 


def dict_or_empty(x):
    return x if isinstance(x, dict) else {}

def flatten_dict_column(df, col, prefix):
    """
    Flattens a column of dicts (possibly containing nested dicts) into top-level columns.
    - df[col] remains untouched; you can drop it afterwards if you like.
    - Returns a new DataFrame with flattened columns prefixed.
    """
    # Ensure every row is a dict (avoid NaN/None)
    series = df[col].apply(dict_or_empty)

    # Use json_normalize with a separator to flatten nested dicts (e.g., Ambience.romantic)
    flat = pd.json_normalize(series, sep="__")
    if not flat.empty:
        flat.columns = [f"{prefix}{c}" for c in flat.columns]
    else:
        # No data -> empty DF with no columns
        flat = pd.DataFrame(index=df.index)
    return flat

# --- Flatten nested dict columns ------------------------------------------

attr_flat  = flatten_dict_column(biz, "attributes", "attr_")
hours_flat = flatten_dict_column(biz, "hours",      "hours_")

# If some attributes values are JSON-encoded strings (rare), try to parse them:
for c in attr_flat.columns:
    # Parse strings that look like mini-JSON dicts (e.g., "{'romantic': False, ...}")
    # Yelp sometimes uses single quotes; replace with double quotes safely when it seems dict-like.
    mask = attr_flat[c].apply(lambda v: isinstance(v, str) and v.strip().startswith("{") and v.strip().endswith("}"))
    if mask.any():
        def try_parse(v):
            if not isinstance(v, str): return v
            s = v.strip()
            # best-effort: convert single quotes to double quotes for JSON parsing
            s_json = s.replace("'", '"')
            try:
                parsed = json.loads(s_json)
                if isinstance(parsed, dict):
                    return parsed
            except Exception:
                pass
            return v
        parsed_series = attr_flat.loc[mask, c].apply(try_parse)
        # If we actually parsed dicts, expand them
        if parsed_series.apply(lambda x: isinstance(x, dict)).any():
            sub = pd.json_normalize(parsed_series, sep="__")
            sub.columns = [f"{c}__{k}" for k in sub.columns]  # keep source col name
            # align indices and join
            attr_flat = attr_flat.drop(columns=[c]).join(sub, how="left")

# --- Categories handling ---------------------------------------------------
# Yelp 'categories' is usually a comma-separated string. Normalize to a clean list or string.
def normalize_categories(val):
    if pd.isna(val):
        return []
    if isinstance(val, list):
        # Occasionally already a list
        return [x.strip() for x in val if isinstance(x, str)]
    # Treat as comma-separated
    return [x.strip() for x in str(val).split(",") if x.strip()]

cats_list = biz["categories"].apply(normalize_categories)

# (Option A) Keep as semicolon-separated string for single-row-per-business
cats_str = cats_list.apply(lambda lst: "; ".join(lst) if lst else None)

# --- Build flat table ------------------------------------------------------
cols_to_drop = ["attributes", "hours", "categories"]
base = biz.drop(columns=[c for c in cols_to_drop if c in biz.columns])

biz_flat = pd.concat([base, attr_flat, hours_flat], axis=1)
biz_flat["categories_norm"] = cats_str

# Optional: sort columns (ID first, readable order)
id_cols = [c for c in ["business_id", "name", "city", "state", "postal_code"] if c in biz_flat.columns]
other_cols = [c for c in biz_flat.columns if c not in id_cols]
biz_flat = biz_flat[id_cols + other_cols]

# 2) Save one-row-per-business CSV
out_csv = RAW / "1_yelp_business_flat.csv"
biz_flat.to_csv(out_csv, index=False)
print(f"Saved: {out_csv}")

print("Shape:", biz.shape)   # (rows, columns)




Saved: ../data/raw/1_yelp_business_flat.csv
Shape: (150346, 15)


## load & filter

In [80]:
from pathlib import Path
import pandas as pd

RAW = Path("../data/raw")
biz = pd.read_csv(RAW / "1_yelp_business_flat.csv")

# keep restaurants only (recommended)
is_rest = biz["categories_norm"].fillna("").str.contains("Restaurants", case=False)
biz = biz.loc[is_rest].copy()

print(biz.shape)
biz.head()


# drop leakage: DO NOT use business.stars as a feature
biz = biz.drop(columns=[c for c in ["stars"] if c in biz.columns])

/var/folders/l_/bfn4zmvj11j3519tmtl4xcd40000gn/T/ipykernel_23648/713055227.py:5: DtypeWarning: Columns (44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  biz = pd.read_csv(RAW / "1_yelp_business_flat.csv")


(52268, 56)


In [81]:
print("Shape:", biz.shape)       # rows, columns
print("Columns:", biz.columns)   # column names
display(biz.head())              # first 5 rows

# Basic checks
biz.info()                       # dtypes + non-null counts
biz.describe()                   # summary stats for numeric cols

print("\nMissing values per column:")
print(biz.isna().sum())

Shape: (52268, 55)
Columns: Index(['business_id', 'name', 'city', 'state', 'postal_code', 'address',
       'latitude', 'longitude', 'review_count', 'is_open',
       'review_count_log1p', 'attr_ByAppointmentOnly',
       'attr_BusinessAcceptsCreditCards', 'attr_BikeParking',
       'attr_RestaurantsPriceRange2', 'attr_CoatCheck',
       'attr_RestaurantsTakeOut', 'attr_RestaurantsDelivery', 'attr_Caters',
       'attr_WiFi', 'attr_WheelchairAccessible', 'attr_HappyHour',
       'attr_OutdoorSeating', 'attr_HasTV', 'attr_RestaurantsReservations',
       'attr_DogsAllowed', 'attr_Alcohol', 'attr_GoodForKids',
       'attr_RestaurantsAttire', 'attr_RestaurantsTableService',
       'attr_RestaurantsGoodForGroups', 'attr_DriveThru', 'attr_NoiseLevel',
       'attr_GoodForMeal', 'attr_BusinessAcceptsBitcoin', 'attr_Smoking',
       'attr_GoodForDancing', 'attr_AcceptsInsurance', 'attr_BestNights',
       'attr_BYOB', 'attr_Corkage', 'attr_BYOBCorkage',
       'attr_HairSpecializesIn', 'attr

,business_id,name,city,state,postal_code,address,latitude,longitude,review_count,is_open,...,attr_AgesAllowed,attr_DietaryRestrictions,hours_Monday,hours_Tuesday,hours_Wednesday,hours_Thursday,hours_Friday,hours_Saturday,hours_Sunday,categories_norm
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,19107,935 Race St,39.955505,-75.155564,80.0,1,...,NaN,NaN,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0,Restaurants; Food; Bubble Tea; Coffee & Tea; B...
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,Ashland City,TN,37015,615 S Main St,36.269593,-87.058943,6.0,1,...,NaN,NaN,0:0-0:0,6:0-22:0,6:0-22:0,6:0-22:0,9:0-0:0,9:0-22:0,8:0-22:0,Burgers; Fast Food; Sandwiches; Food; Ice Crea...
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,Affton,MO,63123,8025 Mackenzie Rd,38.565165,-90.321087,19.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pubs; Restaurants; Italian; Bars; American (Tr...
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,Nashville,TN,37207,2312 Dickerson Pike,36.208102,-86.768170,10.0,1,...,NaN,NaN,0:0-0:0,6:0-21:0,6:0-21:0,6:0-16:0,6:0-16:0,6:0-17:0,6:0-21:0,Ice Cream & Frozen Yogurt; Fast Food; Burgers;...
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,Tampa Bay,FL,33602,NaN,27.955269,-82.456320,10.0,1,...,NaN,NaN,11:0-14:0,11:0-14:0,11:0-14:0,11:0-14:0,11:0-14:0,5:0-10:0,15:0-18:0,Vietnamese; Food; Restaurants; Food Trucks


<class 'pandas.core.frame.DataFrame'>
Index: 52268 entries, 3 to 150340
Data columns (total 55 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_id                      52268 non-null  object 
 1   name                             52268 non-null  object 
 2   city                             52268 non-null  object 
 3   state                            52268 non-null  object 
 4   postal_code                      52247 non-null  object 
 5   address                          51825 non-null  object 
 6   latitude                         52268 non-null  float64
 7   longitude                        52268 non-null  float64
 8   review_count                     52268 non-null  float64
 9   is_open                          52268 non-null  int64  
 10  review_count_log1p               52268 non-null  float64
 11  attr_ByAppointmentOnly           3345 non-null   object 
 12  attr_BusinessAcceptsCr

In [82]:
biz.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52268 entries, 3 to 150340
Data columns (total 55 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_id                      52268 non-null  object 
 1   name                             52268 non-null  object 
 2   city                             52268 non-null  object 
 3   state                            52268 non-null  object 
 4   postal_code                      52247 non-null  object 
 5   address                          51825 non-null  object 
 6   latitude                         52268 non-null  float64
 7   longitude                        52268 non-null  float64
 8   review_count                     52268 non-null  float64
 9   is_open                          52268 non-null  int64  
 10  review_count_log1p               52268 non-null  float64
 11  attr_ByAppointmentOnly           3345 non-null   object 
 12  attr_BusinessAcceptsCr

## Drop Rare or sparse attributes (very low coverage):

In [83]:
from pathlib import Path
import pandas as pd
import re
from datetime import datetime
import numpy as np



# --- Build time features from hours_* strings ---
TIME_FMT = "%H:%M"
RANGE_RE = re.compile(r"^\s*(\d{1,2}:\d{2})\s*-\s*(\d{1,2}:\d{2})\s*$")

def hours_from_range(s):
    if not isinstance(s, str):
        return 0.0
    m = RANGE_RE.match(s)
    if not m:
        return 0.0
    start_str, end_str = m.groups()
    start = datetime.strptime(start_str, TIME_FMT)
    end   = datetime.strptime(end_str, TIME_FMT)
    if start_str == end_str:
        return 24.0 if start_str in ("00:00", "0:00") else 0.0
    if end <= start:  # crosses midnight
        return (24 - (start.hour + start.minute/60)) + (end.hour + end.minute/60)
    return (end - start).seconds / 3600.0

day_names = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
for d in day_names:
    src = f"hours_{d}"
    dst = f"openhours_{d}"
    if src in biz.columns:
        biz[dst] = biz[src].apply(hours_from_range)
    else:
        biz[dst] = 0.0

open_cols = [f"openhours_{d}" for d in day_names]

biz["total_weekly_hours"] = biz[open_cols].sum(axis=1)
biz["days_open"]          = (biz[open_cols] > 0).sum(axis=1)
biz["weekend_hours"]      = biz["openhours_Saturday"] + biz["openhours_Sunday"]
biz["avg_daily_hours"]    = (biz["total_weekly_hours"] / biz["days_open"].replace(0, pd.NA)).fillna(0.0)

# (optional) keep a flag to distinguish missing hours info later, if needed
biz["has_hours_info"]     = (biz[open_cols].sum(axis=1) > 0).astype("int8")

# --- Cast & (optionally) fill, same style as review_count ---
for c in ["total_weekly_hours","days_open","weekend_hours","avg_daily_hours"]:
    biz[c] = pd.to_numeric(biz[c], errors="coerce").astype("float32")
    # If you want to keep NaN for “unknown”, comment the next line out:
    # biz[c] = biz[c].fillna(0)

# --- Now drop leakage + sparse columns (but DO NOT drop the new features) ---
leak_and_sparse = [
    "stars", "name", "postal_code", "address",
    "attr_CoatCheck", "attr_GoodForMeal", "attr_BusinessAcceptsBitcoin",
    "attr_GoodForDancing", "attr_AcceptsInsurance", "attr_BestNights",
    "attr_BYOB", "attr_Corkage", "attr_BYOBCorkage", "attr_HairSpecializesIn",
    "attr_RestaurantsCounterService", "attr_AgesAllowed", "attr_DietaryRestrictions", "attr_Open24Hours",
    # you may drop raw hours_* strings now that features are built:
    "hours_Monday","hours_Tuesday","hours_Wednesday","hours_Thursday",
    "hours_Friday","hours_Saturday","hours_Sunday",
]
biz = biz.drop(columns=[c for c in leak_and_sparse if c in biz.columns])

# --- Save with time features included ---
out_path = RAW / "2_yelp_business_flat_attr_clean.csv"
biz.to_csv(out_path, index=False)
print("Saved:", out_path, "| Shape:", biz.shape)


/var/folders/l_/bfn4zmvj11j3519tmtl4xcd40000gn/T/ipykernel_23648/2804800455.py:42: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  biz["avg_daily_hours"]    = (biz["total_weekly_hours"] / biz["days_open"].replace(0, pd.NA)).fillna(0.0)


Saved: ../data/raw/2_yelp_business_flat_attr_clean.csv | Shape: (52268, 43)


In [84]:
# from pathlib import Path
# import pandas as pd

# RAW = Path("../data/raw")
# biz = pd.read_csv(RAW / "1_yelp_business_flat.csv")

# # keep restaurants only (recommended)
# is_rest = biz["categories_norm"].fillna("").str.contains("Restaurants", case=False)
# biz = biz.loc[is_rest].copy()

# print("Shape before drops:", biz.shape)

# # --- Drop leakage (target column) ---
# biz = biz.drop(columns=[c for c in ["stars"] if c in biz.columns])

# # --- Drop sparse / redundant attributes ---
# cols_to_drop = [
#     "name", "postal_code", "address",
#     "attr_CoatCheck", "attr_GoodForMeal", "attr_BusinessAcceptsBitcoin", 
#     "attr_GoodForDancing", "attr_AcceptsInsurance", "attr_BestNights",
#     "attr_BYOB", "attr_Corkage", "attr_BYOBCorkage", "attr_HairSpecializesIn",
#     "attr_RestaurantsCounterService", "attr_AgesAllowed", "attr_DietaryRestrictions", "attr_Open24Hours",
#     "hours_Monday", "hours_Tuesday", "hours_Wednesday", "hours_Thursday",
#     "hours_Friday", "hours_Saturday", "hours_Sunday",
# ]

# # Drop only those columns that exist
# to_drop = [c for c in cols_to_drop if c in biz.columns]
# biz = biz.drop(columns=to_drop)

# print("Dropped columns:", to_drop)
# print("Shape after drops:", biz.shape)

# # Quick peek
# biz.head()

# out_path = RAW / "2_yelp_business_flat_attr_clean.csv"
# biz.to_csv(out_path, index=False)
# print("Saved:", out_path)


## Attribute : consistent categorical
consistent categorical values that can be used for ML

Flattens dict-like attributes (e.g., attr_GoodForMeal, attr_Ambience, attr_BusinessParking) into subcolumns like attr_GoodForMeal__breakfast.

Normalizes boolean-ish values to {1.0, 0.0, NaN} (Float32).

Normalizes WiFi / Alcohol / NoiseLevel / RestaurantsAttire to consistent categories.

Casts RestaurantsPriceRange2 to numeric 1–4 (Float32).

Fixes junk like u'free', "none'", casual', etc.

In [85]:
# --- Now drop leakage 
drop_week = [
   "openhours_Monday", "openhours_Tuesday", "openhours_Wednesday", "openhours_Thursday", "openhours_Friday", "openhours_Saturday", "openhours_Sunday"
]
biz = biz.drop(columns=[c for c in drop_week if c in biz.columns])

# --- Save with time features included ---
out_path = RAW / "2_yelp_business_drop_week.csv"
biz.to_csv(out_path, index=False)
print("Saved:", out_path, "| Shape:", biz.shape)

Saved: ../data/raw/2_yelp_business_drop_week.csv | Shape: (52268, 36)


In [86]:
from pathlib import Path
import pandas as pd, numpy as np, json, ast, re



# ------------------------
# Helpers
# ------------------------
def parse_dictish(v):
    """Convert dict-like strings to dicts safely; otherwise {}."""
    if isinstance(v, dict):
        return v
    if pd.isna(v):
        return {}
    s = str(v).strip()
    # fix odd trailing quote like "none'"
    if s.endswith("'") and s.count("'") % 2 == 1:
        s = s[:-1]
    # python literal (handles single quotes / True/False/None)
    try:
        out = ast.literal_eval(s)
        if isinstance(out, dict):
            return out
    except Exception:
        pass
    # JSON fallback
    s_json = (s.replace("'", '"')
                .replace("None", "null")
                .replace("True", "true")
                .replace("False", "false"))
    try:
        out = json.loads(s_json)
        if isinstance(out, dict):
            return out
    except Exception:
        pass
    return {}

def norm_token(v):
    """Lowercase, strip quotes/unicode prefixes; return None for empty/none-ish."""
    if pd.isna(v):
        return None
    s = str(v).strip()
    s = s.replace("u'", "'")
    s = s.strip("'").strip('"').strip()
    s_low = s.lower()
    if s_low in {"", "none", "null", "nan"}:
        return None
    return s_low

def to_TFN(v):
    """Map many boolean spellings to 'TRUE'/'FALSE'/NaN (string categories)."""
    t = norm_token(v)
    if t is None:
        return np.nan
    if t in {"true","t","yes","y","1"}:
        return "TRUE"
    if t in {"false","f","no","n","0"}:
        return "FALSE"
    return np.nan  # unexpected values → NaN so they don't pollute categories

# ------------------------
# 1) Expand dict-like attribute columns
# ------------------------
attr_cols = [c for c in biz.columns if c.startswith("attr_")]

dict_like = []
for c in attr_cols:
    ser = biz[c].dropna().astype(str).str.strip()
    if not ser.empty and ser.str.match(r"^\{.*\}$").any():
        dict_like.append(c)

for c in dict_like:
    parsed = biz[c].apply(parse_dictish)
    sub = pd.json_normalize(parsed, sep="__")
    # Name subkeys like attr_GoodForMeal__breakfast, attr_BusinessParking__garage, etc.
    sub.columns = [f"{c}__{k}" for k in sub.columns]
    biz = biz.drop(columns=[c]).join(sub)

# Recompute attribute columns after expansion
attr_cols = [c for c in biz.columns if c.startswith("attr_")]

# ------------------------
# 2) Clean special categorical attributes
# ------------------------
if "attr_WiFi" in biz.columns:
    wifi = biz["attr_WiFi"].apply(norm_token)
    wifi = wifi.map({"no":"no","free":"free","paid":"paid"}).astype("category")
    biz["attr_WiFi"] = wifi

if "attr_Alcohol" in biz.columns:
    alc = biz["attr_Alcohol"].apply(norm_token)
    # normalize common variants
    alc = alc.replace({
        "full_bar":"full_bar",
        "full bar":"full_bar",
        "beer_and_wine":"beer_and_wine",
        "beer and wine":"beer_and_wine",
        "none":"none"
    })
    valid_alc = {"none","beer_and_wine","full_bar"}
    alc = alc.where(alc.isin(valid_alc))
    biz["attr_Alcohol"] = alc.astype("category")

if "attr_RestaurantsAttire" in biz.columns:
    attire = biz["attr_RestaurantsAttire"].apply(norm_token)
    attire = attire.replace({"casual":"casual","dressy":"dressy","formal":"formal"})
    attire = attire.where(attire.isin({"casual","dressy","formal"}))
    biz["attr_RestaurantsAttire"] = attire.astype("category")

if "attr_NoiseLevel" in biz.columns:
    noise = biz["attr_NoiseLevel"].apply(norm_token)
    noise = noise.where(noise.isin({"quiet","average","loud","very_loud"}))
    # keep as categorical; you can also map to ordered codes later if you prefer an ordinal
    biz["attr_NoiseLevel"] = noise.astype("category")

if "attr_AgesAllowed" in biz.columns:
    ages = biz["attr_AgesAllowed"].apply(norm_token)
    # leave as categorical (values like '21plus','allages' appear)
    biz["attr_AgesAllowed"] = ages.astype("category")

# ------------------------
# 3) Clean all remaining boolean-ish attribute columns to TRUE/FALSE/NaN
# ------------------------
# We exclude the special categorical ones we just handled above
special_cats = {"attr_WiFi","attr_Alcohol","attr_RestaurantsAttire","attr_NoiseLevel","attr_AgesAllowed"}
boolish_attr = [c for c in attr_cols if c not in special_cats]

for c in boolish_attr:
    # If column looks numeric-only (e.g. price range), skip boolean mapping
    if pd.api.types.is_numeric_dtype(biz[c]):
        continue
    # Apply TRUE/FALSE/NaN mapping
    biz[c] = biz[c].apply(to_TFN).astype("category")

# ------------------------
# 4) (Optional) Numeric cast for a few known numeric attrs
# ------------------------
for c in ["attr_RestaurantsPriceRange2"]:
    if c in biz.columns:
        biz[c] = pd.to_numeric(biz[c], errors="coerce").astype("Float32")

print("Done cleaning attributes.")
print("Example columns and categories:")
for c in ["attr_WiFi","attr_Alcohol","attr_RestaurantsAttire","attr_NoiseLevel"]:
    if c in biz.columns:
        print(c, "→", list(biz[c].cat.categories) if pd.api.types.is_categorical_dtype(biz[c]) else "not categorical")

# Save a cleaned copy (optional)
out = RAW / "3_yelp_business_flat_attr_formatted.csv"
biz.to_csv(out, index=False)
print("Saved:", out)

biz.shape

Done cleaning attributes.
Example columns and categories:
attr_WiFi → ['free', 'no', 'paid']
attr_Alcohol → ['beer_and_wine', 'full_bar']
attr_RestaurantsAttire → ['casual', 'dressy', 'formal']
attr_NoiseLevel → ['average', 'loud', 'quiet', 'very_loud']


/var/folders/l_/bfn4zmvj11j3519tmtl4xcd40000gn/T/ipykernel_23648/62268177.py:147: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  print(c, "→", list(biz[c].cat.categories) if pd.api.types.is_categorical_dtype(biz[c]) else "not categorical")


Saved: ../data/raw/3_yelp_business_flat_attr_formatted.csv


(52268, 36)

## Categories (Top 20)

In [87]:
# --- Top-20 categories → tri-state features (1/0/NaN) ---

import re
from collections import Counter
from pathlib import Path
import numpy as np
import pandas as pd

# 0) Ensure we’re marking restaurants
is_rest = biz["categories_norm"].fillna("").str.contains(r"\bRestaurants\b", case=False, regex=True)

# 1) Split categories_norm into unique, trimmed labels (handle ';' or ',')
def split_cats(val):
    if pd.isna(val): 
        return None
    parts = [p.strip() for p in re.split(r"[;,]", str(val)) if p.strip()]
    return sorted(set(parts)) if parts else None

cats_parsed = biz["categories_norm"].map(split_cats)

# 2) Filter out umbrella / non-food verticals for counting
drop_umbrella = {
    "Restaurants","Food","Nightlife","Bars","Local Services","Shopping",
    "Event Planning & Services","Active Life","Hotels & Travel","Health & Medical",
    "Beauty & Spas","Automotive","Home Services","Arts & Entertainment",
    "Professional Services"
}
drop_non_food = {
    "Pets","Pet Services","Real Estate","Doctors","Dentists","Auto Repair",
    "Home & Garden","Fashion","Hair Salons","Nail Salons","Medical Centers",
    "Education","Lawyers","Financial Services","Contractors"
}

# 3) Count among restaurants only
counts = Counter()
for lst in cats_parsed[is_rest].dropna():
    for c in lst:
        if c not in drop_umbrella and c not in drop_non_food:
            counts[c] += 1

# 4) Keep Top-N deterministically
TOP_N = 20
kept_cats = [c for c, _ in sorted(counts.items(), key=lambda kv: (-kv[1], kv[0]))[:TOP_N]]
print("Top categories kept:", kept_cats)

# (optional) save coverage table
cov = (pd.Series(counts, name="count")
         .reindex(kept_cats)
         .to_frame()
         .assign(coverage_pct=lambda d: 100 * d["count"] / is_rest.sum()))
Path("../data/processed").mkdir(parents=True, exist_ok=True)
cov.to_csv("../data/processed/top20_categories_coverage.csv", index_label="category")

# 5) Build tri-state features: 1 present, 0 absent, NaN if categories missing for that row
cat_feats = pd.DataFrame(index=biz.index)
for k in kept_cats:
    cat_feats[f"cat__{k}"] = cats_parsed.map(
        lambda lst: np.nan if lst is None else (1.0 if k in lst else 0.0)
    ).astype("Float32")

# 6) Join to your DataFrame (and optionally drop the raw text column)
biz = pd.concat([biz, cat_feats], axis=1)
biz = biz.drop(columns=["categories_norm"])  # uncomment if you no longer need the raw text

print("biz shape after cat features:", biz.shape)

# Save a cleaned copy (optional)
out = RAW / "4_yelp_business_top_category.csv"
biz.to_csv(out, index=False)
print("Saved:", out)
biz.shape

Top categories kept: ['Sandwiches', 'American (Traditional)', 'Pizza', 'Fast Food', 'Breakfast & Brunch', 'American (New)', 'Burgers', 'Mexican', 'Italian', 'Coffee & Tea', 'Seafood', 'Chinese', 'Salad', 'Chicken Wings', 'Cafes', 'Delis', 'Caterers', 'Specialty Food', 'Bakeries', 'Desserts']
biz shape after cat features: (52268, 55)
Saved: ../data/raw/4_yelp_business_top_category.csv


(52268, 55)

## City

Normalizes city + state.

Builds city_slim (Top-20 vs “Other”).

One-hot encodes into city_* columns.

Drops the helper columns (city_clean, city_slim) so your modeling table only keeps the dummies.

Saves the Top-20 list for reproducibility.

In [88]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- US state codes ---
US_STATES = {
    'AL','AK','AZ','AR','CA','CO','CT','DC','DE','FL','GA','HI','IA','ID','IL','IN','KS','KY',
    'LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','OH',
    'OK','OR','PA','RI','SC','SD','TN','TX','UT','VA','VT','WA','WI','WV','WY'
}

# --- normalize city/state ---
norm_map = {
    "St. Louis": "Saint Louis",
    "St Louis": "Saint Louis",
    "St. Petersburg": "Saint Petersburg",
    "St Petersburg": "Saint Petersburg",
}
biz["city_clean"] = biz["city"].astype(str).str.strip().replace(norm_map)
biz["state"] = biz["state"].astype(str).str.strip().str.upper()

# --- compute Top-20 US cities ---
us_mask = biz["state"].isin(US_STATES)
top20_us_cities = (
    biz.loc[us_mask, "city_clean"]
       .value_counts()
       .head(20)
       .index
)

# --- collapse others to "Other" ---
biz["city_slim"] = biz["city_clean"].where(biz["city_clean"].isin(top20_us_cities), "Other")
biz["city_slim"] = biz["city_slim"].fillna("Other")

# --- create dummies ---
city_ohe = pd.get_dummies(biz["city_slim"], prefix="city", dtype=np.float32)
biz = pd.concat([biz, city_ohe], axis=1)

# --- drop helper cols ---
cols_to_drop = [c for c in ["city_clean", "city_slim"] if c in biz.columns]
biz = biz.drop(columns=cols_to_drop)

print("City dummies added:", list(city_ohe.columns))
print("Shape after city encoding:", biz.shape)


out = RAW / "5_yelp_business_top_city.csv" 
biz.to_csv(out, index=False) 
print("Saved:", out) 
biz.shape

City dummies added: ['city_Boise', 'city_Brandon', 'city_Cherry Hill', 'city_Clearwater', 'city_Franklin', 'city_Indianapolis', 'city_Largo', 'city_Meridian', 'city_Metairie', 'city_Nashville', 'city_New Orleans', 'city_Other', 'city_Philadelphia', 'city_Reno', 'city_Saint Louis', 'city_Saint Petersburg', 'city_Santa Barbara', 'city_Sparks', 'city_Tampa', 'city_Tucson', 'city_Wilmington']
Shape after city encoding: (52268, 76)
Saved: ../data/raw/5_yelp_business_top_city.csv


(52268, 76)

## Create Meta

In [89]:

import json
import pandas as pd
from pandas.api.types import (
    is_numeric_dtype,
    is_categorical_dtype,
    is_object_dtype,
    is_datetime64_any_dtype,
)

def _safe_nunique(s: pd.Series) -> int:
    """nunique that tolerates unhashable objects like dict/list/set."""
    try:
        return s.nunique(dropna=True)
    except TypeError:
        # Convert unhashables to a stable string form then count
        return (
            s.dropna()
             .apply(lambda x: json.dumps(x, sort_keys=True) if isinstance(x, (dict, list, set)) else x)
             .nunique(dropna=True)
        )

def _safe_samples(s: pd.Series, k: int = 5):
    """Return up to k sample values, robust to unhashables."""
    try:
        # If hashable, use unique() for a quick look
        vals = s.dropna().head(1000).unique()
        return list(vals[:k])
    except TypeError:
        # For unhashables (dict/list/set), stringify a few rows
        return [json.dumps(v, sort_keys=True) if isinstance(v, (dict, list, set)) else str(v)
                for v in s.dropna().head(k)]

def create_metadata(df: pd.DataFrame) -> pd.DataFrame:
    meta = pd.DataFrame(index=df.columns)

    # Core stats
    meta["dtype"]       = df.dtypes.astype(str)
    meta["n_unique"]    = [ _safe_nunique(df[c]) for c in df.columns ]
    meta["missing_sum"] = df.isna().sum()

    # Type flags
    meta["is_numeric"]     = [is_numeric_dtype(df[c]) for c in df.columns]
    meta["is_categorical"] = [is_categorical_dtype(df[c]) for c in df.columns]
    meta["is_string"]      = [is_object_dtype(df[c]) for c in df.columns]  # note: includes dicts too
    meta["is_datetime"]    = [is_datetime64_any_dtype(df[c]) for c in df.columns]

    # Min / Max only for numeric or datetime
    mins, maxs = {}, {}
    for c in df.columns:
        try:
            if is_datetime64_any_dtype(df[c]) or is_numeric_dtype(df[c]):
                mins[c] = df[c].min()
                maxs[c] = df[c].max()
            else:
                mins[c], maxs[c] = None, None
        except Exception:
            mins[c], maxs[c] = None, None
    meta["min_value"] = pd.Series(mins)
    meta["max_value"] = pd.Series(maxs)

    # Sample values (robust to unhashables)
    samples = { c: _safe_samples(df[c], k=5) for c in df.columns }
    meta["sample_values"] = pd.Series(samples)

    return meta

In [90]:

meta_business = create_metadata(biz)

/var/folders/l_/bfn4zmvj11j3519tmtl4xcd40000gn/T/ipykernel_23648/1688460008.py:43: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  meta["is_categorical"] = [is_categorical_dtype(df[c]) for c in df.columns]


In [91]:
# Save metadata DataFrame to CSV
from pathlib import Path
PROCESSED = Path("../data/processed"); PROCESSED.mkdir(parents=True, exist_ok=True)
meta_business.to_csv(PROCESSED / "6_business_metadata_report.csv", index=True)
print("Saved to", PROCESSED / "6_business_metadata_report.csv")

Saved to ../data/processed/6_business_metadata_report.csv


# Review Dataset

In [92]:
from pathlib import Path
import pandas as pd

RAW = Path("../data/raw")
review_df_2019 = pd.read_json(RAW / "yelp_reviews_2019_latest.json", lines=True)

print(review_df_2019.shape)       # rows, columns
print(review_df_2019.columns)     # column names
review_df_2019.info()


(2111695, 9)
Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111695 entries, 0 to 2111694
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 145.0+ MB


In [93]:
# from pathlib import Path
# import pandas as pd
# from IPython.display import display  # for display()

# RAW = Path("../data/raw")
# rev_path = RAW / "yelp_reviews_2019_latest.json"  # change to .json.gz if needed
# assert rev_path.exists(), f"File not found: {rev_path}"

# read_kwargs = dict(lines=True)
# if rev_path.suffix == ".gz":
#     read_kwargs["compression"] = "gzip"

# review_df_2019 = pd.read_json(rev_path, **read_kwargs)

# print("Loaded from:", rev_path)
# print("Shape:", review_df_2019.shape)
# print("Columns:", review_df_2019.columns.tolist())
# display(review_df_2019.head())

# # make sure 'date' is datetime
# review_df_2019["date"] = pd.to_datetime(review_df_2019["date"], errors="coerce")

# print("\nMissing values per column:\n", review_df_2019.isna().sum())
# print("\nNumeric summary:\n", review_df_2019.select_dtypes("number").describe())

# # (optional) quick year sanity check
# if "date" in review_df_2019:
#     yrs = review_df_2019["date"].dt.year.dropna()
#     if not yrs.empty:
#         print("\nYear range:", int(yrs.min()), "→", int(yrs.max()))





In [94]:
file_path = "../data/raw/yelp_reviews_2019_latest.json"

with open(file_path, "r", encoding="utf-8") as f:
    row_count = sum(1 for _ in f)

print("Total rows:", row_count)

Total rows: 2111712


In [95]:
# Function: Coerce integer-encoded features to categorical if few unique values
def int_to_categorical(df, max_unique_values=10):
    df_copy = df.copy()
    for col in df_copy.select_dtypes(include=["int64", "int32"]).columns:
        n_unique = df_copy[col].nunique()
        if n_unique <= max_unique_values:
            df_copy[col] = df_copy[col].astype("category")
    return df_copy

# Apply to review dataset
review2019_df_cat = int_to_categorical(review_df_2019)
print("\nAfter converting some int columns to category:")
review2019_df_cat.info()


After converting some int columns to category:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111695 entries, 0 to 2111694
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        category      
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(3), object(4)
memory usage: 130.9+ MB


## Create metadata review 2019

In [96]:
review2019_metadata_report = create_metadata(review2019_df_cat)

/var/folders/l_/bfn4zmvj11j3519tmtl4xcd40000gn/T/ipykernel_23648/1688460008.py:43: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  meta["is_categorical"] = [is_categorical_dtype(df[c]) for c in df.columns]


In [97]:
# Save metadata DataFrame to CSV
from pathlib import Path
PROCESSED = Path("../data/processed")
review2019_metadata_report.to_csv(PROCESSED / "7_review2019_metadata_report.csv", index=True)
print("Saved to", PROCESSED / "7_review2019_metadata_report.csv")


Saved to ../data/processed/7_review2019_metadata_report.csv


## Aggregate review 2019

In [98]:
# import pandas as pd
# import numpy as np

# # --- 1) Aggregate 2019 reviews to business-level targets ---
# rev_tmp = review2019_df_cat[["business_id", "stars", "date"]].copy()
# rev_tmp["stars_num"] = pd.to_numeric(rev_tmp["stars"], errors="coerce")
# rev_tmp["date"]      = pd.to_datetime(rev_tmp["date"], errors="coerce")

# agg19 = (rev_tmp.dropna(subset=["stars_num"])
#                .groupby("business_id", as_index=False)
#                .agg(
#                    rev_count_2019=("stars_num","size"),
#                    avg_stars_2019=("stars_num","mean"),
#                    first_review_2019=("date","min"),
#                    last_review_2019=("date","max"),
#                ))

# # Optional: keep only businesses with enough 2019 reviews
# MIN_REV = 3     # use 1/3/5 depending on coverage you want
# agg19 = agg19[agg19["rev_count_2019"] >= MIN_REV].copy()

# # Set dtypes for NEW cols only (does NOT touch business_df dtypes)
# agg19["rev_count_2019"] = agg19["rev_count_2019"].astype("int32")
# agg19["avg_stars_2019"] = agg19["avg_stars_2019"].astype("float32")

# # --- 2) Merge with your business features (keeps business_df dtypes) ---
# assert biz["business_id"].is_unique
# Xy19 = biz.merge(agg19, on="business_id", how="inner", validate="one_to_one")
# print("Joined rows:", len(Xy19))
# display(Xy19.head())






## add review word length 

rl_share_long200w is one of the aggregate features you created when you added review length signals at the business level.

Let’s break it down:

rl → shorthand for review length

share → proportion (a fraction between 0 and 1)

long200w → reviews that are “long,” defined as having ≥ 200 words

So:

\text{rl\_share\_long200w for business X} \;=\; \frac{\text{# of reviews with ≥200 words for business X in 2019}}{\text{total # of 2019 reviews for business X}}

Example:

If a restaurant had 20 reviews in 2019

Out of those, 5 reviews had ≥200 words

Then rl_share_long200w = 5 / 20 = 0.25

In [99]:
# --- Start from your review df (already loaded as review2019_df_cat) ---
rev_tmp = review2019_df_cat[["business_id", "stars", "date", "text"]].copy()
rev_tmp["stars_num"] = pd.to_numeric(rev_tmp["stars"], errors="coerce")
rev_tmp["date"]      = pd.to_datetime(rev_tmp["date"], errors="coerce")

# --- Per-review length (words) and short flag ---
text_s = rev_tmp["text"].astype(str)                                # guard against NaN
rev_tmp["len_word"] = text_s.str.count(r"\b\w+\b").astype("Int32")  # word count
rev_tmp["is_short24"] = (rev_tmp["len_word"] <= 24).astype("Int8")  # short review flag

# --- Aggregate to business level (add the two features you want) ---
agg19 = (
    rev_tmp.dropna(subset=["stars_num"])
           .groupby("business_id", as_index=False)
           .agg(
               rev_count_2019    = ("stars_num","size"),
               avg_stars_2019    = ("stars_num","mean"),
               first_review_2019 = ("date","min"),
               last_review_2019  = ("date","max"),

               # the two new features:
               rl_word_mean      = ("len_word","mean"),
               rl_share_short24  = ("is_short24","mean"),   # <-- consistent name
           )
)

# --- Stability filter & dtypes (same as your pipeline) ---
MIN_REV = 3
agg19 = agg19.loc[agg19["rev_count_2019"] >= MIN_REV].copy()

agg19["rev_count_2019"]   = agg19["rev_count_2019"].astype("int32")
agg19["avg_stars_2019"]   = agg19["avg_stars_2019"].astype("float32")
agg19["rl_word_mean"]     = agg19["rl_word_mean"].astype("float32")
agg19["rl_share_short24"] = agg19["rl_share_short24"].astype("float32")

# --- Merge with business features (unchanged) ---
assert biz["business_id"].is_unique
Xy19 = biz.merge(agg19, on="business_id", how="inner", validate="one_to_one")




In [100]:
# Save BEFORE EDA & BEFORE IMPUTATION (correct)
PROC = Path("../data/processed"); PROC.mkdir(parents=True, exist_ok=True)
csv_path = PROC / "8_biz_merged_2019.csv"
Xy19.to_csv(csv_path, index=False)
print("Saved:", csv_path)


Saved: ../data/processed/8_biz_merged_2019.csv


In [101]:
# If you want just min and max:
print("Min:", Xy19["rl_word_mean"].min())
print("Max:", Xy19["rl_word_mean"].max())

# Or use describe() for more stats:
print(Xy19["rl_word_mean"].describe())


Min: 18.333334
Max: 540.5
count    36261.000000
mean        88.404472
std         28.092167
min         18.333334
25%         70.428574
50%         85.000000
75%        102.000000
max        540.500000
Name: rl_word_mean, dtype: float64


In [102]:
# Compute per-review word counts
rev_tmp["len_word"] = rev_tmp["text"].astype(str).str.count(r"\b\w+\b")

# Check distribution & find 90th percentile
long_cutoff = rev_tmp["len_word"].quantile(0.90)
short_cutoff = rev_tmp["len_word"].quantile(0.10)

print("10th percentile (short):", short_cutoff)
print("90th percentile (long):", long_cutoff)


10th percentile (short): 24.0
90th percentile (long): 200.0


short cut off is 10% cut off

In [103]:
Xy19.columns  

Index(['business_id', 'city', 'state', 'latitude', 'longitude', 'review_count',
       'is_open', 'review_count_log1p', 'attr_ByAppointmentOnly',
       'attr_BusinessAcceptsCreditCards', 'attr_BikeParking',
       'attr_RestaurantsPriceRange2', 'attr_RestaurantsTakeOut',
       'attr_RestaurantsDelivery', 'attr_Caters', 'attr_WiFi',
       'attr_WheelchairAccessible', 'attr_HappyHour', 'attr_OutdoorSeating',
       'attr_HasTV', 'attr_RestaurantsReservations', 'attr_DogsAllowed',
       'attr_Alcohol', 'attr_GoodForKids', 'attr_RestaurantsAttire',
       'attr_RestaurantsTableService', 'attr_RestaurantsGoodForGroups',
       'attr_DriveThru', 'attr_NoiseLevel', 'attr_Smoking',
       'total_weekly_hours', 'days_open', 'weekend_hours', 'avg_daily_hours',
       'has_hours_info', 'cat__Sandwiches', 'cat__American (Traditional)',
       'cat__Pizza', 'cat__Fast Food', 'cat__Breakfast & Brunch',
       'cat__American (New)', 'cat__Burgers', 'cat__Mexican', 'cat__Italian',
       'cat__Cof

## Normalize

Normalize text (case/spacing/variants)

Map your specified attributes to consistent numeric codes

Keep missing values as <NA> (nullable integers)

Save the result to ../data/processed/biz_merge.csv

I followed your exact mappings:

Booleans (False, True) → 1, 2

attr_WiFi: free, no, paid → 1, 2, 3

attr_RestaurantsAttire: casual, formal, dressy → 1, 2, 3

attr_NoiseLevel: average, quiet, loud, very loud → 1, 2, 3, 4 (your order)

attr_Alcohol: full bar, beer and wine → 1, 2

If a value doesn’t match the mapping after normalization, it becomes <NA>.

In [104]:
import pandas as pd
import numpy as np
import re

# ----------------- helpers -----------------
def norm_token(x):
    """Lowercase, trim, collapse whitespace/underscores; None for empty/none-ish."""
    if pd.isna(x):
        return None
    s = str(x).strip().strip("'\"")
    s = s.replace("u'", "'")
    s = s.replace("_", " ")
    s = re.sub(r"\s+", " ", s).lower().strip()
    if s in {"", "none", "null", "nan"}:
        return None
    return s

def map_bool_012(x):
    """NaN/unknown -> 0, falsey -> 1, truey -> 2."""
    t = norm_token(x)
    if t is None:
        return 0
    if t in {"false","f","no","n","0"}:
        return 1
    if t in {"true","t","yes","y","1"}:
        return 2
    # raw numerics that slipped through
    if t.isdigit():
        v = int(t)
        if v == 0: return 1
        if v == 1: return 2
    return 0  # unexpected -> treat as unknown

# ----------------- columns -----------------
bool_cols = [
    'attr_ByAppointmentOnly','attr_BusinessAcceptsCreditCards','attr_BikeParking',
    'attr_RestaurantsTakeOut','attr_RestaurantsDelivery','attr_Caters',
    'attr_WheelchairAccessible','attr_HappyHour','attr_OutdoorSeating','attr_HasTV',
    'attr_RestaurantsReservations','attr_DogsAllowed','attr_GoodForKids',
    'attr_RestaurantsTableService','attr_RestaurantsGoodForGroups','attr_DriveThru',
    'attr_Smoking'
]

# mappings for multi-class attrs (include common variants)
wifi_map = {"free": 1, "no": 2, "paid": 3}
attire_map = {"casual": 1, "formal": 2, "dressy": 3}
noise_map = {"average": 1, "quiet": 2, "loud": 3, "very loud": 4, "very_loud": 4}
alcohol_map = {
    "full bar": 1, "fullbar": 1, "full_bar": 1,
    "beer and wine": 2, "beer_and_wine": 2, "beer & wine": 2, "beer and wind": 2  # handles the typo
}

# ----------------- apply to Xy19 -----------------
df = Xy19.copy()

# 1) boolean-like → 0/1/2
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].map(map_bool_012).astype("Int8")

# 2) RestaurantsPriceRange2 → 0–4 (0 for missing)
if "attr_RestaurantsPriceRange2" in df.columns:
    pr = pd.to_numeric(df["attr_RestaurantsPriceRange2"], errors="coerce")
    pr = pr.where(pr.isin([1,2,3,4]))
    df["attr_RestaurantsPriceRange2"] = pr.fillna(0).astype("float32")

# 3) WiFi → 0(no value)/1/2/3
if "attr_WiFi" in df.columns:
    df["attr_WiFi"] = df["attr_WiFi"].map(lambda v: wifi_map.get(norm_token(v), 0)).astype("float32")

# 4) RestaurantsAttire → 0/1/2/3
if "attr_RestaurantsAttire" in df.columns:
    df["attr_RestaurantsAttire"] = df["attr_RestaurantsAttire"].map(
        lambda v: attire_map.get(norm_token(v), 0)
    ).astype("float32")

# 5) NoiseLevel → 0/1/2/3/4
if "attr_NoiseLevel" in df.columns:
    df["attr_NoiseLevel"] = df["attr_NoiseLevel"].map(
        lambda v: noise_map.get(norm_token(v), 0)
    ).astype("float32")

# 6) Alcohol → 0/1/2
if "attr_Alcohol" in df.columns:
    df["attr_Alcohol"] = df["attr_Alcohol"].map(
        lambda v: alcohol_map.get(norm_token(v), 0)
    ).astype("float32")

# ----------------- (optional) quick sanity check -----------------
check_cols = (
    bool_cols
    + [c for c in ["attr_RestaurantsPriceRange2","attr_WiFi","attr_RestaurantsAttire","attr_NoiseLevel","attr_Alcohol"] if c in df.columns]
)
for c in check_cols:
    vc = df[c].value_counts(dropna=False).sort_index()
    print(f"{c}:")
    print(vc, "\n")

# Now df has consistent numeric codes for the attributes you listed


# Save
OUT = Path("../data/processed") / "9_biz_merged_normalized.csv"
OUT.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT, index=False)
print("Saved:", OUT)


attr_ByAppointmentOnly:
attr_ByAppointmentOnly
0    33122
1     3032
2      107
Name: count, dtype: Int64 

attr_BusinessAcceptsCreditCards:
attr_BusinessAcceptsCreditCards
0     4889
1      812
2    30560
Name: count, dtype: Int64 

attr_BikeParking:
attr_BikeParking
0     9408
1     6942
2    19911
Name: count, dtype: Int64 

attr_RestaurantsTakeOut:
attr_RestaurantsTakeOut
0     3074
1     1323
2    31864
Name: count, dtype: Int64 

attr_RestaurantsDelivery:
attr_RestaurantsDelivery
0     4898
1     8590
2    22773
Name: count, dtype: Int64 

attr_Caters:
attr_Caters
0    10665
1    10248
2    15348
Name: count, dtype: Int64 

attr_WheelchairAccessible:
attr_WheelchairAccessible
0    24474
1      847
2    10940
Name: count, dtype: Int64 

attr_HappyHour:
attr_HappyHour
0    25391
1     4256
2     6614
Name: count, dtype: Int64 

attr_OutdoorSeating:
attr_OutdoorSeating
0     7938
1    14250
2    14073
Name: count, dtype: Int64 

attr_HasTV:
attr_HasTV
0     6543
1     5692
2    2402

## This file creates merge business file and review files 

We build our dataset from the Yelp Open Dataset (2024 mirror), focusing on food venues identified through the categories field and excluding the stars column to prevent label leakage. From business.json, we derive structured features: (i) numerical variables such as review_count, its log transform, is_open, latitude, longitude, and weekly_open_hours (parsed from JSON); (ii) selected high-coverage attributes (e.g., RestaurantsReservations, OutdoorSeating, BusinessAcceptsCreditCards, BikeParking, GoodForKids, RestaurantsDelivery, RestaurantsTakeOut, WheelchairAccessible, HasTV, DogsAllowed), represented as binary indicators with missing values preserved as NaN; (iii) categorical indicators for the top 20 cuisine/venue labels after removing umbrella tags, encoded as 1 (present), 0 (absent), or NaN (if the entire field is missing), along with an optional missingness flag; and (iv) geographic features, including normalized city names and one-hot encodings of the 20 most common U.S. cities, with other cities collapsed into “Other.”

We then aggregate reviews from a pre-extracted 2019 subset to compute business-level targets: average star rating, review count, and first/last review dates, applying a minimum threshold of three reviews per business. Check-in records are similarly summarized as totals, unique days, and temporal spans. All features are merged on business_id to form the modeling table (biz_merged_2019.csv). Missing values are deliberately preserved and handled later in the modeling pipeline (e.g., imputing binary features to 0, continuous features to the median or group-wise statistics). Numeric variables are stored as float32, while identifiers and dates are retained only for joins and reporting.


## Review Dataset

We use reviews from the Yelp Open Dataset (2019 subset). The raw review file (`yelp_reviews_2019_latest.json`) is loaded in JSON lines format, with gzip compression support if required. We ensure the `date` field is parsed as a datetime object, and we check for missing values and basic statistics. Integer-encoded fields with few unique values are optionally coerced to categorical type for efficiency. We preserve missing values in the raw review data, deferring imputation and handling to the modeling pipeline.  

We create a metadata report for the review dataset to summarize column data types, unique value counts, missingness, and sample values. At the business level, we aggregate reviews by `business_id` to compute labels and activity signals: total 2019 review count (`rev_count_2019`), average 2019 star rating (`avg_stars_2019`), and first/last review dates (`first_review_2019`, `last_review_2019`). We apply a stability rule requiring `MIN_REV ≥ 3` reviews per business to ensure reliable targets. Aggregated review features are stored as float32 for consistency.  

The resulting business-level aggregates are merged with the engineered business feature table on `business_id` to produce the final modeling dataset (`biz_merged_2019.csv`). Identifiers and dates are retained only for joins and reporting, not as model inputs. 



### To quick count where losing rows

In [105]:
from pathlib import Path
import pandas as pd
import numpy as np

RAW = Path("../data/raw")
PROC = Path("../data/processed")

# --- 1) Business table with categories_norm ---
# Use the flattened CSV you saved earlier (adjust name if yours differs)
biz = pd.read_csv(RAW / "1_yelp_business_flat.csv")

# Simple sanity check
assert "business_id" in biz.columns, "business_id missing in biz"
assert "categories_norm" in biz.columns, "categories_norm missing in biz"

# --- 2) Raw business row count (from JSON) ---
biz0 = pd.read_json(RAW / "yelp_academic_dataset_business.json", lines=True)
print("business.json rows:", len(biz0))

# --- 3) Filters: Restaurants-only vs Restaurants+Food ---
is_restaurants_only = biz["categories_norm"].fillna("").str.contains(r"\bRestaurants\b", case=False, regex=True)

# broader scope similar to your classmate’s description
is_food_or_rest = biz["categories_norm"].fillna("").str.contains(
    r"\b(Restaurants|Food|Cafes|Bakeries|Coffee & Tea|Delis|Desserts)\b",
    case=False, regex=True
)

print("after Restaurants-only filter:", int(is_restaurants_only.sum()))
print("after Restaurants+Food filter:", int(is_food_or_rest.sum()))

# choose which scope to use downstream:
biz_scoped = biz.loc[is_restaurants_only].copy()       # OR use is_food_or_rest

# --- 4) Build 2019 review aggregate with MIN_REV >= 3 ---
rev_path = RAW / "yelp_reviews_2019_latest.json"   # update if your file name differs
read_kwargs = dict(lines=True)
if rev_path.suffix == ".gz":
    read_kwargs["compression"] = "gzip"

reviews = pd.read_json(rev_path, **read_kwargs)
reviews["stars_num"] = pd.to_numeric(reviews["stars"], errors="coerce")
reviews["date"]      = pd.to_datetime(reviews["date"], errors="coerce")

agg19 = (reviews.dropna(subset=["stars_num"])
                 .groupby("business_id", as_index=False)
                 .agg(
                     rev_count_2019=("stars_num","size"),
                     avg_stars_2019=("stars_num","mean"),
                     first_review_2019=("date","min"),
                     last_review_2019=("date","max"),
                 ))

MIN_REV = 3
agg19 = agg19.loc[agg19["rev_count_2019"] >= MIN_REV].copy()

print("unique business_ids in agg19:", agg19["business_id"].nunique())

# --- 5) Join type effect ---
m_inner = biz_scoped.merge(agg19, on="business_id", how="inner", validate="one_to_one")
m_left  = biz_scoped.merge(agg19, on="business_id", how="left")

print("inner join rows:", len(m_inner))
print("left  join rows:", len(m_left), " (NaN targets:", m_left["avg_stars_2019"].isna().sum(), ")")

# Optional: save the inner-join modeling table
PROC.mkdir(parents=True, exist_ok=True)
m_inner.to_csv(PROC / "biz_merged_2019_inner.csv", index=False)


/var/folders/l_/bfn4zmvj11j3519tmtl4xcd40000gn/T/ipykernel_23648/2955945496.py:10: DtypeWarning: Columns (44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  biz = pd.read_csv(RAW / "1_yelp_business_flat.csv")


business.json rows: 150346
after Restaurants-only filter: 52268
after Restaurants+Food filter: 64616


/var/folders/l_/bfn4zmvj11j3519tmtl4xcd40000gn/T/ipykernel_23648/2955945496.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_food_or_rest = biz["categories_norm"].fillna("").str.contains(


unique business_ids in agg19: 97861
inner join rows: 36261
left  join rows: 52268  (NaN targets: 16007 )


In [106]:
print("Restaurants in scope:", len(biz_scoped))
print("With ≥3 reviews (inner):", len(m_inner))
print("Dropped due to no target:", len(biz_scoped) - len(m_inner))


Restaurants in scope: 52268
With ≥3 reviews (inner): 36261
Dropped due to no target: 16007


## Which join to use?

Inner join
✅ Best for supervised modeling (predicting ratings, finding feature importances).
You only keep businesses where the target (average stars) is actually defined.
→ That’s why you see ~36k rows.

Left join
Useful if you want to keep the entire restaurant universe for EDA or for later prediction on new businesses with no reviews yet.
But those 16k rows with NaN labels can’t be used for training supervised models.

Outer join
Rarely useful here — would just add even more NaNs.
